In [ ]:
!pip install fastapi uvicorn
!pip install langchain langchain-community
!pip install chromadb
!pip install transformers
!pip install accelerate
!pip install huggingface_hub
!pip install langchain-google-genai
!pip install google-generativeai
!pip install html2text

In [ ]:
import requests
import json
import re
from bs4 import BeautifulSoup
import html2text

In [ ]:
import requests
import json
from pprint import pprint

# 노션 API 설정
NOTION_TOKEN = "your_integration_token_here"
DATABASE_ID = "your_database_id_here"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"  # 현재 최신 API 버전
}

def get_database_content():
    """노션 데이터베이스의 내용을 가져옵니다."""
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    response = requests.post(url, headers=headers)
    data = response.json()

    # API 응답 확인
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(data)
        return None

    return data["results"]

def get_page_content(page_id):
    """특정 페이지의 내용을 가져옵니다."""
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"

    response = requests.get(url, headers=headers)
    data = response.json()

    # API 응답 확인
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(data)
        return None

    return data["results"]

def extract_text_content(blocks):
    """블록에서 텍스트 내용을 추출합니다."""
    content = []

    for block in blocks:
        block_type = block["type"]

        if block_type == "paragraph":
            text_content = ""
            for text in block["paragraph"]["rich_text"]:
                text_content += text["plain_text"]
            if text_content:
                content.append(text_content)

        elif block_type == "heading_1":
            text_content = ""
            for text in block["heading_1"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"# {text_content}")

        elif block_type == "heading_2":
            text_content = ""
            for text in block["heading_2"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"## {text_content}")

        elif block_type == "heading_3":
            text_content = ""
            for text in block["heading_3"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"### {text_content}")

        elif block_type == "bulleted_list_item":
            text_content = ""
            for text in block["bulleted_list_item"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"• {text_content}")

        elif block_type == "numbered_list_item":
            text_content = ""
            for text in block["numbered_list_item"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"1. {text_content}")  # 실제 번호는 표시 시 조정 필요

        elif block_type == "to_do":
            text_content = ""
            for text in block["to_do"]["rich_text"]:
                text_content += text["plain_text"]
            checked = "✓" if block["to_do"]["checked"] else "☐"
            content.append(f"{checked} {text_content}")

        elif block_type == "code":
            text_content = ""
            for text in block["code"]["rich_text"]:
                text_content += text["plain_text"]
            language = block["code"]["language"]
            content.append(f"```{language}\n{text_content}\n```")

        elif block_type == "quote":
            text_content = ""
            for text in block["quote"]["rich_text"]:
                text_content += text["plain_text"]
            content.append(f"> {text_content}")

    return content

def main():
    # 데이터베이스의 모든 페이지 가져오기
    database_pages = get_database_content()
    if not database_pages:
        print("데이터베이스를 가져오는데 실패했습니다.")
        return

    print(f"총 {len(database_pages)}개의 페이지를 찾았습니다.")

    for page in database_pages:
        # 페이지 제목 가져오기 (페이지 속성에 따라 조정 필요)
        page_id = page["id"]

        # 제목 추출 시도 - 페이지 속성에 따라 달라질 수 있음
        title = None

        # 일반적인 제목 속성 확인
        if "properties" in page:
            for prop_name, prop_value in page["properties"].items():
                if prop_value["type"] == "title" and prop_value["title"]:
                    title_parts = [text["plain_text"] for text in prop_value["title"]]
                    title = "".join(title_parts)
                    break

        if title:
            print(f"\n페이지: {title}")
        else:
            print(f"\n페이지 ID: {page_id}")

        # 페이지 내용 가져오기
        page_blocks = get_page_content(page_id)
        if not page_blocks:
            print("페이지 내용을 가져오는데 실패했습니다.")
            continue

        # 텍스트 내용 추출
        content = extract_text_content(page_blocks)

        # 결과 출력
        print("내용:")
        for line in content:
            print(line)

        # 파일로 저장 (선택적)
        if title:
            filename = f"{title.replace(' ', '_')}.txt"
            with open(filename, "w", encoding="utf-8") as f:
                for line in content:
                    f.write(line + "\n")
            print(f"파일 저장됨: {filename}")

In [ ]:
if __name__ == "__main__":
    main()